In [1]:
####################################################################################
# Import Packages
####################################################################################
# Localhost
# http://localhost:4040/ 

# Packages
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

import os
import sys
import pyspark
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, lit, col
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql.functions import rank
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.types import StructType,StructField 
from pyspark.sql.types import StringType, IntegerType, ArrayType
from pyspark.sql.types import StringType, ArrayType,StructType,StructField


spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

In [2]:
import pandas as pd
titanic = pd.read_csv("/Users/largo/Google Drive/DS/Python/5_Importing_Data_1/Data/titanic_sub.csv", sep=",", header=0)
titanic[['Sex', 'Ticket', "Cabin", "Embarked"]] = titanic[['Sex', 'Ticket', "Cabin", "Embarked"]].astype(str)
titanic = spark.createDataFrame(titanic)
titanic.show()

+-----------+--------+------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|  male|22.0|    1|    0|       A/5 21171|   7.25|  nan|       S|
|          2|       1|     1|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|female|26.0|    0|    0|STON/O2. 3101282|  7.925|  nan|       S|
|          4|       1|     1|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|  male|35.0|    0|    0|          373450|   8.05|  nan|       S|
|          6|       0|     3|  male| NaN|    0|    0|          330877| 8.4583|  nan|       Q|
|          7|       0|     1|  male|54.0|    0|    0|           17463|51.8625|  E46|       S|
|          8|       0|     3|  male| 2.0|    3|    1|       

In [3]:
titanic.schema

StructType(List(StructField(PassengerId,LongType,true),StructField(Survived,LongType,true),StructField(Pclass,LongType,true),StructField(Sex,StringType,true),StructField(Age,DoubleType,true),StructField(SibSp,LongType,true),StructField(Parch,LongType,true),StructField(Ticket,StringType,true),StructField(Fare,DoubleType,true),StructField(Cabin,StringType,true),StructField(Embarked,StringType,true)))

In [64]:
# Remap the values in "Pclass" to Proletarian/Ambitious/Burgouise OK
# Remap the values in "Sex" to 0/1 OK
# Take log of "Age" OK
# Clean the column "Fare" for inflation OK
# Extract the number from the column "Ticket"
# Fill in the missing values in the Cabin column
from utils import genderselector
from pyspark.sql.types import StringType, BooleanType, DateType

def return_function(data, prefix, gender):
    df = data
    to_drop = ["PassengerId", "SibSp"]
    df = df.drop(*to_drop)
    df = df.withColumn("Class",  when(df.Pclass == 3, "Prol").when(df.Pclass == 2, "Ambitious").when(df.Pclass == 1, "Burgeous"))
    df = df.withColumn("Sex", when(df.Sex=="male", "MALE").when(df.Sex=="female", "FEMALE"))
    df = df.withColumn("Fare", df.Fare*100)
    df = df.withColumn("Parch_New", F.when(F.col("Parch")==0, F.array(F.lit('666'))))
    df = df.withColumn("Parch_New", F.when(F.col("Parch")==0, F.array(F.lit("666"), F.lit("777"))))


    # Write a function, add a column, if Parch_New=666 then "first" otherwise "second"

    # Dictionary Loop to recast DataTypes
    column_types = {
        "StringType()": ["Sex", "Ticket", "Cabin", "Embarked"],
        "IntegerType()": ["Pclass", "Age", "SibSp", "Parch"],
        "BooleanTyp0()": ["Survived"]
        }

    for column in df.columns:
        if column in column_types.items():
            df = df.withColumn("item", df.item.cast("key"))

    """
        for key, value in age_group.items():
        if df.Age in value:
            df = df[key] 
    """

    #df = df.withColumn("Survived", df.Survived.cast(BooleanType()))
    #df = df.withColumn("Sex", df.Sex.cast(StringType()))
    #df = df.withColumn("Age", df.Age.cast(IntegerType()))



    # Write a loop that recasts the column types
    string_typo = ["Sex", "Ticket", "Cabin", "Embarked"]
    if string_typo in df.columns:
        for i in string_typo:
            df = df.withColumn(i, df.i.cast("String"))    
    

    

    # A gender selector implemented as a function
    # df = genderselector(dataframe=df, selector="MALE")
    return df
 

    # This renames the columns
    for name in df.columns:
        if prefix not in df.columns:
            df = df.withColumnRenamed(name, name + '_' + prefix)
        else:
            raise ValueError('The chosen prefix already appers in the dataframe.')
    

    return df

new_df = return_function(titanic, "Y", "MALE")
new_df.show(truncate=False)





+--------+------+------+----+-----+----------------+-----------------+-----+--------+---------+----------+
|Survived|Pclass|Sex   |Age |Parch|Ticket          |Fare             |Cabin|Embarked|Class    |Parch_New |
+--------+------+------+----+-----+----------------+-----------------+-----+--------+---------+----------+
|0       |3     |MALE  |22.0|0    |A/5 21171       |725.0            |nan  |S       |Prol     |[666, 777]|
|1       |1     |FEMALE|38.0|0    |PC 17599        |7128.33          |C85  |C       |Burgeous |[666, 777]|
|1       |3     |FEMALE|26.0|0    |STON/O2. 3101282|792.5            |nan  |S       |Prol     |[666, 777]|
|1       |1     |FEMALE|35.0|0    |113803          |5310.0           |C123 |S       |Burgeous |[666, 777]|
|0       |3     |MALE  |35.0|0    |373450          |805.0000000000001|nan  |S       |Prol     |[666, 777]|
|0       |3     |MALE  |NaN |0    |330877          |845.8299999999999|nan  |Q       |Prol     |[666, 777]|
|0       |1     |MALE  |54.0|0    |17

In [45]:
new_df = new_df.withColumn("CONCAT", F.concat(new_df.Sex, F.lit(" "), new_df.Class).alias("CONCAT"))
new_df.show()

+--------+------+------+----+-----+----------------+-----------------+-----+--------+---------+----------+----------------+
|Survived|Pclass|   Sex| Age|Parch|          Ticket|             Fare|Cabin|Embarked|    Class| Parch_New|          CONCAT|
+--------+------+------+----+-----+----------------+-----------------+-----+--------+---------+----------+----------------+
|       0|     3|  MALE|22.0|    0|       A/5 21171|            725.0|  nan|       S|     Prol|[666, 777]|       MALE Prol|
|       1|     1|FEMALE|38.0|    0|        PC 17599|          7128.33|  C85|       C| Burgeous|[666, 777]| FEMALE Burgeous|
|       1|     3|FEMALE|26.0|    0|STON/O2. 3101282|            792.5|  nan|       S|     Prol|[666, 777]|     FEMALE Prol|
|       1|     1|FEMALE|35.0|    0|          113803|           5310.0| C123|       S| Burgeous|[666, 777]| FEMALE Burgeous|
|       0|     3|  MALE|35.0|    0|          373450|805.0000000000001|  nan|       S|     Prol|[666, 777]|       MALE Prol|
|       

In [48]:
window_sep = Window.partitionBy("Embarked").orderBy("Fare")
new_df.withColumn("RANK", F.rank().over(window_sep)).show()


+--------+------+------+----+-----+----------+-----------------+-----+--------+-----+----------+-----------+----+
|Survived|Pclass|   Sex| Age|Parch|    Ticket|             Fare|Cabin|Embarked|Class| Parch_New|     CONCAT|RANK|
+--------+------+------+----+-----+----------+-----------------+-----+--------+-----+----------+-----------+----+
|       0|     3|  MALE|19.0|    0|    365222|            675.0|  nan|       Q| Prol|[666, 777]|  MALE Prol|   1|
|       0|     3|FEMALE|18.0|    0|    365226|            675.0|  nan|       Q| Prol|[666, 777]|FEMALE Prol|   1|
|       0|     3|  MALE| NaN|    0|    394140|685.8299999999999|  nan|       Q| Prol|[666, 777]|  MALE Prol|   3|
|       0|     3|  MALE| NaN|    0|    368323|            695.0|  nan|       Q| Prol|[666, 777]|  MALE Prol|   4|
|       0|     3|FEMALE| NaN|    0|    330909|           762.92|  nan|       Q| Prol|[666, 777]|FEMALE Prol|   5|
|       0|     3|  MALE| NaN|    0|     36209|            772.5|  nan|       Q| Prol|[66

In [63]:
window_sep = Window.partitionBy("SEX").orderBy("Ticket")
new_df = new_df.withColumn("RANK", F.rank().over(window_sep))
new_df.filter(new_df["RANK"]==1).show()

+--------+------+------+----+-----+------+-----------------+-----+--------+--------+----------+----+
|Survived|Pclass|   Sex| Age|Parch|Ticket|             Fare|Cabin|Embarked|   Class| Parch_New|RANK|
+--------+------+------+----+-----+------+-----------------+-----+--------+--------+----------+----+
|       0|     1|  MALE|52.0|    1|110413|7965.000000000001|  E67|       S|Burgeous|      null|   1|
|       1|     1|FEMALE|30.0|    0|110152|           8650.0|  B77|       S|Burgeous|[666, 777]|   1|
|       1|     1|FEMALE|16.0|    0|110152|           8650.0|  B79|       S|Burgeous|[666, 777]|   1|
|       1|     1|FEMALE|33.0|    0|110152|           8650.0|  B77|       S|Burgeous|[666, 777]|   1|
+--------+------+------+----+-----+------+-----------------+-----+--------+--------+----------+----+



In [38]:
new_df = new_df.select(new_df.Ticket, F.explode(new_df.Parch_New))
new_df.show()

+----------------+---+
|          Ticket|col|
+----------------+---+
|       A/5 21171|666|
|       A/5 21171|777|
|        PC 17599|666|
|        PC 17599|777|
|STON/O2. 3101282|666|
|STON/O2. 3101282|777|
|          113803|666|
|          113803|777|
|          373450|666|
|          373450|777|
|          330877|666|
|          330877|777|
|           17463|666|
|           17463|777|
|          237736|666|
|          237736|777|
|          113783|666|
|          113783|777|
|       A/5. 2151|666|
|       A/5. 2151|777|
+----------------+---+
only showing top 20 rows



In [40]:
titanic.printSchema()

root
 |-- PassengerId: long (nullable = true)
 |-- Survived: long (nullable = true)
 |-- Pclass: long (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: long (nullable = true)
 |-- Parch: long (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [ ]:
import pandas as pd
titanic = pd.read_csv("/Users/largo/Google Drive/DS/Python/5_Importing_Data_1/Data/titanic_sub.csv", sep=",", header=0)
titanic[['Sex', 'Ticket', "Cabin", "Embarked"]] = titanic[['Sex', 'Ticket', "Cabin", "Embarked"]].astype(str)
titanic = titanic.iloc[0:4, :]

def return_function(data):
    # This does some random stuff
    df = data

    # Create age buckets
    age_group = {
        "young": range(0,25),
        "middleaged": range(26,50),
        "old": range(51,110)
        }


    for key, value in age_group.items():
        if df.Age in value:
            df = df[key]
    
    return df

new_df = return_function(titanic)
print(new_df)

In [84]:
simpleData = [("James", "Sales", 3000),
    ("Michael", "Sales", 4600),
    ("Robert", "Sales", 4100),
    ("Kumar", "Marketing", 2000),
    ("Saif", "Sales", 4100)]
schema = ["employee_name", "department", "salary"]
table = spark.createDataFrame(data=simpleData, schema=schema)

def recasting_function(data):
    df = data

    column_types = {
        "StringType()": ["employee_name", "department"],
        "IntegerType()": ["salary"]
        }

    for column in df.columns:
        if column in column_types.items():
            df = df.withColumn(item, df.item.cast(key))
    
    return df
        

result = recasting_function(table)

In [85]:
result.printSchema()

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: long (nullable = true)



In [89]:
# This creates a sample dataframe
simpleData = [("James", "Sales", 3000),
    ("Michael", "Sales", 4600),
    ("Robert", "Sales", 4100),
    ("Kumar", "Marketing", 2000),
    ("Saif", "Sales", 4100)]
schema = ["employee_name", "department", "salary"]
table = spark.createDataFrame(data=simpleData, schema=schema)

# This is the function which is supposed to change datatypes en bulk
def recasting_function(data):
    column_types = {
        "string": ["employee_name", "department", "salary"]
        }
    for (k, v) in column_types.items():
        for c in v:
            if c in data.columns:
                data = data.withColumn(c, data[c].cast(k))    
    return data
        
# Here I apply it to my sample dataset
result = recasting_function(table)

In [90]:
result.printSchema()

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: string (nullable = true)

